In [22]:
import numpy as np
import pandas as pd
import yfinance as yf
import datetime
import time
import csv
from PY_OZNAL_UTILS.oznal_utils import OZNAL_UTIL


In [23]:

def loadCommentData(fromFS=False):


    if fromFS:
        df = pd.read_csv("data_end/post_2.csv")
        df['date'] = pd.to_datetime(df['date'], format="%Y-%m-%d")
        df = df.drop(columns=['Unnamed: 0'])
        return df

    df = pd.read_csv('data_spark/posts_2.csv')
    df = df.drop(columns=['_c0','Unnamed: 0','body','id'])
    df = df.rename(columns={"created_utc": "date", "total_awards_received" : "awards"})
    df['score'] = df['score'].fillna(1)
    df['awards'] =  df['awards'].fillna(0)
    df = df.dropna()
    df = df.astype({'score': 'int32', 'awards':'int32'})
    df['date'] = pd.to_datetime(df['date'], format="%Y-%m-%d %H:%M:%S")
    df['date'] = df['date'].dt.strftime('%Y-%m-%d')
    df.to_csv("data_spark/posts_2_1.csv")

        # read saved dataset
    with open('data_spark/posts_2_1.csv', mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        line_count = 0
        # create new dataset
        with open('data_spark/post_2_1_new.csv', mode='w') as newFile:
            newCsv = csv.writer(newFile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL )
            newCsv.writerow(['symbols', 'score', 'date','awards'])
            for row in csv_reader:
                symbols = row['symbols'].replace('[', '').replace(']', '').replace('\'', '').split(',')
                score = row['score']
                date = row['date']
                awards = row['awards']
                for symbol in symbols:
                    newCsv.writerow([symbol, score, date, awards])

        ###########################################################

        # read created dataset
        df = pd.read_csv('data_spark/post_2_1_new.csv')
        # cast types
        df['date'] = pd.to_datetime(df['date'], format="%Y-%m-%d")
        # merge date with symbol
        df['concatColumn'] = df.apply(lambda x : x['symbols'] + " " + str(x['date']), axis=1)
        df = df.groupby('concatColumn').sum()
        # unmerge date and symbol
        df['symbol'] = ""
        df['date'] = ""
        for i, value in enumerate(df.index):
            df['symbol'][i] = str(value).strip().split(" ")[0]
            df['date'][i] =  str(value).strip().split(" ")[1]
        df.index = range(0, len(df))
        #cast date
        df['date'] = pd.to_datetime(df['date'], format="%Y-%m-%d")
        # unique symbols
        uniqueSymbols = list(df['symbol'].unique()) 
        df = df.sort_values(by='date')
        start = df['date'].loc[1]
        end =  df['date'].iloc[-1]
        print(start,end)
        ###########################################################

        # stock_final = pd.DataFrame()
        # OZNAL = OZNAL_UTIL()
        # symbols = OZNAL.getStockSymbols()
        # for symbol in uniqueSymbols:
        #     if symbol in  ['BABY', 'LONG', 'GAME', 'ACAT','ADA','AMDA','ARCI', 'ARCI','BBRY', 'BNB']:
        #         continue
        #     try:
        #         x = OZNAL.downloadNASDAQHistoricalStockData(symbol,start,end)
        #         x['date'] = x.index
        #         result = pd.merge(df, x, on=["symbol","date"])
        #         stock_final = stock_final.append(result)
        #     except:
        #         pass

        df.to_csv("data_end/post_2.csv")
        return df

In [24]:
def loadPostData(fromFS=False):

    if fromFS:
        df = pd.read_csv("data_end/post.csv")
        df['date'] = pd.to_datetime(df['date'], format="%Y-%m-%d")
        df = df.drop(columns=['Unnamed: 0'])
        return df

    # load  data set from spark
    df = pd.read_csv('data_spark/posts.csv')
    # drop un used columns
    df = df.drop(columns=['_c0','Unnamed: 0','selftext','id', 'title'])
    # rename columns
    df = df.rename(columns={"created_utc": "date", "num_comments" : "awards" })
    # fill na
    df['score'] = df['score'].fillna(1)
    df['awards'] = df['awards'].fillna(0)
    df['upvote_ratio'] = df['upvote_ratio'].fillna(0)

    # cast columns
    df = df.astype({'score': 'float32', "awards" : "float32"})
    df['date'] = pd.to_datetime(df['date'], format="%Y-%m-%d")
    df['date'] = df['date'].dt.strftime('%Y-%m-%d')
    
    # save daset
    df.to_csv("data_spark/posts_1.csv")

    ###########################################################

    # read saved dataset
    with open('data_spark/posts_1.csv', mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        line_count = 0
        # create new dataset
        with open('data_spark/post_1_new.csv', mode='w') as newFile:
            newCsv = csv.writer(newFile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL )
            newCsv.writerow(['symbols', 'score', 'date','awards','upvote_ratio'])
            for row in csv_reader:
                symbols = row['symbols'].replace('[', '').replace(']', '').replace('\'', '').split(',')
                score = row['score']
                date = row['date']
                awards = row['awards']
                upV = row['upvote_ratio']
                for symbol in symbols:
                    newCsv.writerow([symbol, score, date, awards,upV])

    ###########################################################

    # read created dataset
    df = pd.read_csv('data_spark/post_1_new.csv')
    # cast types
    df['date'] = pd.to_datetime(df['date'], format="%Y-%m-%d")
    # merge date with symbol
    df['concatColumn'] = df.apply(lambda x : x['symbols'] + " " + str(x['date']), axis=1)
    df = df.groupby('concatColumn').sum()
    # unmerge date and symbol
    df['symbol'] = ""
    df['date'] = ""
    for i, value in enumerate(df.index):
        df['symbol'][i] = str(value).strip().split(" ")[0]
        df['date'][i] =  str(value).strip().split(" ")[1]
    df.index = range(0, len(df))
    #cast date
    df['date'] = pd.to_datetime(df['date'], format="%Y-%m-%d")
    # unique symbols
    uniqueSymbols = list(df['symbol'].unique()) 
    df = df.sort_values(by='date')
    start = df['date'].loc[1]
    end =  df['date'].iloc[-1]

    ###########################################################

    # stock_final = pd.DataFrame()
    # OZNAL = OZNAL_UTIL()
    # symbols = OZNAL.getStockSymbols()
    # for symbol in uniqueSymbols:
    #     if symbol in  ['BABY', 'LONG', 'GAME', 'ACAT','ADA','AMDA','ARCI', 'ARCI','BBRY', 'BNB']:
    #         continue
    #     try:
    #         x = OZNAL.downloadNASDAQHistoricalStockData(symbol,start,end)
    #         x['date'] = x.index
    #         result = pd.merge(df, x, on=["symbol","date"])
    #         stock_final = stock_final.append(result)
    #     except:
    #         pass

    df.to_csv("data_end/post.csv")
    return df


In [25]:
%%time
df = loadPostData(fromFS=False)


<ipython-input-24-c14d2114b22f>:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['symbol'][i] = str(value).strip().split(" ")[0]
<ipython-input-24-c14d2114b22f>:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'][i] =  str(value).strip().split(" ")[1]
Wall time: 16.6 s


In [26]:
%%time
df2 = loadCommentData(fromFS=False)

<ipython-input-23-5d76d114139c>:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['symbol'][i] = str(value).strip().split(" ")[0]
<ipython-input-23-5d76d114139c>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'][i] =  str(value).strip().split(" ")[1]
2020-12-12 00:00:00 2021-02-03 00:00:00
Wall time: 17.5 s


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22672 entries, 6956 to 12476
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   score         22672 non-null  float64       
 1   awards        22672 non-null  float64       
 2   upvote_ratio  22672 non-null  float64       
 3   symbol        22672 non-null  object        
 4   date          22672 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 1.5+ MB


In [28]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15017 entries, 7431 to 5840
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   score   15017 non-null  int64         
 1   awards  15017 non-null  int64         
 2   symbol  15017 non-null  object        
 3   date    15017 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 1.1+ MB


In [29]:
result = pd.merge(df, df2, on=["symbol","date"], how="outer")
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34491 entries, 0 to 34490
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   score_x       25502 non-null  float64       
 1   awards_x      25502 non-null  float64       
 2   upvote_ratio  25502 non-null  float64       
 3   symbol        34491 non-null  object        
 4   date          34491 non-null  datetime64[ns]
 5   score_y       17170 non-null  float64       
 6   awards_y      17170 non-null  float64       
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 2.1+ MB


In [34]:
def superFuc(row):
    columnNames = ["score","awards"]
    for columnName in columnNames:
        row[columnName] = row[columnName+"_x"] + row[columnName+"_y"]
    return row

result['score_x'] = result['score_x'].fillna(1)
result['awards_x'] =  result['awards_x'].fillna(0)
result['upvote_ratio'] =  result['upvote_ratio'].fillna(1)
result['score_y'] = result['score_y'].fillna(1)
result['awards_y'] =  result['awards_y'].fillna(0)
dfMerged = result.apply(lambda row: superFuc(row), axis=1)
dfMerged

,score_x,awards_x,upvote_ratio,symbol,date,score_y,awards_y,score,awards
0,1.0,9.0,0.0,BBBY,2019-01-01,1.0,0.0,2.0,9.0
1,1.0,10.0,0.0,NFLX,2019-01-01,1.0,0.0,2.0,10.0
2,1.0,81.0,0.0,ADHD,2019-01-01,1.0,0.0,2.0,81.0
3,1.0,1.0,0.0,CALM,2019-01-01,1.0,0.0,2.0,1.0
4,1.0,3.0,0.0,ATVI,2019-01-01,1.0,0.0,2.0,3.0
...,...,...,...,...,...,...,...,...,...
34486,1.0,0.0,1.0,RAIL,2021-02-03,1.0,0.0,2.0,0.0
34487,1.0,0.0,1.0,ARCC,2021-02-03,4.0,0.0,5.0,0.0
34488,1.0,0.0,1.0,AMKR,2021-02-03,1.0,0.0,2.0,0.0
34489,1.0,0.0,1.0,EBIX,2021-02-03,3.0,0.0,4.0,0.0


In [40]:
stock_final = pd.DataFrame()
uniqueSymbols = list(dfMerged['symbol'].unique())
start = dfMerged['date'].loc[1]
end =  dfMerged['date'].iloc[-1]
OZNAL = OZNAL_UTIL()
symbols = OZNAL.getStockSymbols()
for symbol in uniqueSymbols:
    try:
        x = OZNAL.downloadNASDAQHistoricalStockData(symbol,start,end)
        x['date'] = x.index
        result = pd.merge(df, x, on=["symbol","date"])
        stock_final = stock_final.append(result)
    except:
        pass
stock_final.to_csv("data_end/final.csv")


is date range, symbol may be delisted

1 Failed download:
- FTR: No data found, symbol may be delisted

1 Failed download:
- TAX: No data found, symbol may be delisted

1 Failed download:
- LOGM: No data found, symbol may be delisted

1 Failed download:
- HSGX: No data found, symbol may be delisted

1 Failed download:
- INTL: No data found, symbol may be delisted

1 Failed download:
- PTX: No data found, symbol may be delisted

1 Failed download:
- BWLD: No data found for this date range, symbol may be delisted

1 Failed download:
- WMGI: No data found, symbol may be delisted

1 Failed download:
- DRAM: No data found for this date range, symbol may be delisted

1 Failed download:
- DRYS: No data found, symbol may be delisted

1 Failed download:
- MEET: No data found, symbol may be delisted

1 Failed download:
- ISM: No data found, symbol may be delisted

1 Failed download:
- BDE: No data found for this date range, symbol may be delisted

1 Failed download:
- LOOK: No data found for thi

In [45]:
stock_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14820 entries, 0 to 0
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   score         14820 non-null  float64       
 1   awards        14820 non-null  float64       
 2   upvote_ratio  14820 non-null  float64       
 3   symbol        14820 non-null  object        
 4   date          14820 non-null  datetime64[ns]
 5   Open          14820 non-null  float64       
 6   High          14820 non-null  float64       
 7   Low           14820 non-null  float64       
 8   Close         14820 non-null  float64       
 9   Adj Close     14820 non-null  float64       
 10  Volume        14820 non-null  float64       
dtypes: datetime64[ns](1), float64(9), object(1)
memory usage: 1.6+ MB
